Ho 3 cell-lines, per 2 mi danno tutto e mi hanno anche separato in train and validation, per il terzo mi danno solo le x
In cage_train mi danno le x che sono dove sono i picchi di produzione per ogni gene
Poi mi danno informazioni aggiuntive in DNAsi (quanto è accessibile il DNA), e marker degli histoni (Quelli che incominciano con la H..)
La cosa importante per il risultato e che i numeri sono nell ordine giusto, non importa la magnitudo di questi numeri
Per calcolare l errore usano spearman, che controlla solo in rank

Quello che alleno sul chromosoma 1 va bene anche sul 3, oppure anche se avessero lo stesso code, non sono comparabili

In [2]:
# Import libraries that are required to run your project
# You are allowed to add more libraries as you need

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import torch
import torch.nn as nn

## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [ ]:
def process_CAGE(df_x, df_y):
    df = pd.merge(df_x, df_y, on='gene_name')

    df['middle'] = (df['TSS_start'] + df['TSS_end']) // 2

    df['chr'] = df['chr'].str.replace('chr', '').astype(int)
    df = df.sort_values(by='chr')

    chr_groups = [group for _, group in df.groupby('chr')]
    
    xs, ys = [], []
    for group in chr_groups:
        xs.append(group['middle'].to_numpy())
        ys.append(group['gex'].to_numpy())

    return xs, ys, df['chr'].unique()


In [210]:
#Cage data
train_x1 = pd.read_csv('Data/CAGE-train/X1_train_info.tsv', sep='\t', usecols=[0,1,4,5])
train_y1 = pd.read_csv('Data/CAGE-train/X1_train_y.tsv', sep='\t')
valid_x1 = pd.read_csv('Data/CAGE-train/X1_val_info.tsv', sep='\t', usecols=[0,1,4,5])
valid_y1 = pd.read_csv('Data/CAGE-train/X1_val_y.tsv', sep='\t')
train_x2 = pd.read_csv('Data/CAGE-train/X2_train_info.tsv', sep='\t', usecols=[0,1,4,5])
train_y2 = pd.read_csv('Data/CAGE-train/X2_train_y.tsv', sep='\t')
valid_x2 = pd.read_csv('Data/CAGE-train/X2_val_info.tsv', sep='\t', usecols=[0,1,4,5])
valid_y2 = pd.read_csv('Data/CAGE-train/X2_val_y.tsv', sep='\t')

train_x1 = pd.concat([train_x1, valid_x1])
train_y1 = pd.concat([train_y1, valid_y1])
train_x2 = pd.concat([train_x2, valid_x2])
train_y2 = pd.concat([train_y2, valid_y2])


t_TTS_1, t_y_1, chromosomes = process_CAGE(train_x1, train_y1)
t_TTS_2, t_y_2, _ = process_CAGE(train_x2, train_y2)

0        112658387
1         16738668
2         28225288
3         28931981
4        102452814
           ...    
16279      2389810
16280     58247810
16281     48865532
16282      6481783
16283     13920301
Name: middle, Length: 16284, dtype: int64
0        112658387
1         16738668
2         28225288
3         28931992
4        102452814
           ...    
16279      2389810
16280     58247810
16281     48865532
16282      6481762
16283     13920301
Name: middle, Length: 16284, dtype: int64


In [185]:
#arr[x][y][z][j][k]
#x = Type of histon dnase
#y = which cell type
#z = which chromosome
#j = which peak
#k = which value (0=start, 1=signalValue)
arr = np.empty((7, 3), dtype=object)

In [ ]:
def process_DNAse(df, chromosomes=chromosomes):
    # Assign columns (assuming 4 relevant ones)
    df.columns = ['chr', 'start', 'end', 'value']

    # Compute midpoint (TSS-like)
    df['middle'] = (df['start'] + df['end']) // 2

    # Convert 'chr' to numeric
    def chr_to_num_safe(chr_val):
        try:
            num = int(chr_val.replace('chr', ''))
            return num if num in chromosomes else np.nan
        except (ValueError, AttributeError):
            return np.nan

    df['chr'] = df['chr'].apply(chr_to_num_safe)
    df = df.dropna(subset=['chr'])  # remove non-numeric rows
    df.loc[:, 'chr'] = df['chr'].astype(int)

    # Sort by chromosome
    df = df.sort_values(by='chr')

    # Split into list of DataFrames per chromosome
    chr_groups = [group for _, group in df.groupby('chr')]

    # Convert each to numpy array (middle, value)
    DNAse = [group[['middle', 'value']].to_numpy() for group in chr_groups]

    return DNAse

In [228]:
DNAse_1 = pd.read_csv('Data/bed/DNase-bed/X1.bed', sep='\t', usecols=[0,1,2,6], header=None)
DNAse_2 = pd.read_csv('Data/bed/DNase-bed/X2.bed', sep='\t', usecols=[0,1,2,6], header=None)
DNAse_3 = pd.read_csv('Data/bed/DNase-bed/X3.bed', sep='\t', usecols=[0,1,2,6], header=None)

DNAse_1 = process_DNAse(DNAse_1)
DNAse_2 = process_DNAse(DNAse_2)
DNAse_3 = process_DNAse(DNAse_3)

arr[0] = [DNAse_1, DNAse_2, DNAse_3]

       chr   start     end     value  middle
27551  2.0  193275  193425  9.084758  193350
Empty DataFrame
Columns: [chr, start, end, value, middle]
Index: []
Empty DataFrame
Columns: [chr, start, end, value, middle]
Index: []


In [ ]:
def process_histone(df, chromosomes=chromosomes):
    # Assign columns (assuming 4 relevant ones)
    df.columns = ['chr', 'start', 'value', 'offset']

    # Compute midpoint (TSS-like)
    df['middle'] = df['start'] + df['offset']

    # Convert 'chr' to numeric
    def chr_to_num_safe(chr_val):
        try:
            num = int(chr_val.replace('chr', ''))
            return num if num in chromosomes else np.nan
        except (ValueError, AttributeError):
            return np.nan

    df['chr'] = df['chr'].apply(chr_to_num_safe)
    df = df.dropna(subset=['chr'])  # remove non-numeric rows
    df.loc[:, 'chr'] = df['chr'].astype(int)

    # Sort by chromosome
    df = df.sort_values(by='chr')

    # Split into list of DataFrames per chromosome
    chr_groups = [group for _, group in df.groupby('chr')]

    # Convert each to numpy array (middle, value)
    histone = [group[['middle', 'value']].to_numpy() for group in chr_groups]

    return histone

In [231]:
for i, path in enumerate(['H3K4me1', 'H3K4me3', 'H3K9me3', 'H3K27ac', 'H3K27me3', 'H3K36me3']):
    
    a_1 = pd.read_csv(f'Data/bed/{path}-bed/X1.bed', sep='\t', usecols=[0,1,6,9], header=None)
    a_2 = pd.read_csv(f'Data/bed/{path}-bed/X2.bed', sep='\t', usecols=[0,1,6,9], header=None)
    a_3 = pd.read_csv(f'Data/bed/{path}-bed/X3.bed', sep='\t', usecols=[0,1,6,9], header=None)

    a_1 = process_histone(a_1)
    a_2 = process_histone(a_2)
    a_3 = process_histone(a_3)

    arr[i+1] = [a_1, a_2, a_3]

           start  offset     middle
0      100000253      46  100000299
1      100006554     258  100006812
2      100010185     199  100010384
3      100013131     759  100013890
4      100014884     578  100015462
...          ...     ...        ...
89807    9828835     113    9828948
89808    9829280     245    9829525
89809    9830018     326    9830344
89810     998053     182     998235
89811    9996913     552    9997465

[89812 rows x 3 columns]
            start  offset     middle
0       100000345     133  100000478
1       100006066     235  100006301
2       100009171     126  100009297
3       100010163     158  100010321
4       100020325     113  100020438
...           ...     ...        ...
135471    7103205     211    7103416
135472    7272001     235    7272236
135473    7279333     160    7279493
135474    7320785     345    7321130
135475    7359083      60    7359143

[135476 rows x 3 columns]
            start  offset     middle
0       100003390     140  1000035

In [190]:
N_PEAKS = 5  # number of closest peaks to consider
x = []  # final list: one entry per gene

def find_closest_rows(arr, target, x=N_PEAKS):
    arr = np.asarray(arr)
    diff = np.abs(arr[:, 0] - target)
    idx = np.argsort(diff)[:x]
    return arr[idx]

for chr_index, chr_tss in enumerate(t_TTS_1):  # for each chromosome
    for pos in chr_tss:  # for each TSS position (gene)
        gene_features = [find_closest_rows(marker[0][chr_index], pos) for marker in arr]
        x.append(gene_features)

for chr_index, chr_tss in enumerate(t_TTS_2):  # for each chromosome
    for pos in chr_tss:  # for each TSS position (gene)
        gene_features = [find_closest_rows(marker[1][chr_index], pos) for marker in arr]
        x.append(gene_features)


In [191]:
y = np.concatenate(t_y_1 + t_y_2)

In [192]:
np.savez("train_data.npz", x=x, y=y)

In [193]:
data = np.load("train_data.npz", allow_pickle=True)
x = data["x"]
y = data["y"]

print(x.shape)
print(y.shape)

(32568, 7, 5, 2)
(32568,)


Test

In [194]:
test_x = pd.read_csv('Data/CAGE-train/X3_test_info.tsv', sep='\t', usecols=[0,1,4,5])

## Work Package 1.2 - Model Building

In [195]:
##TRAINING AND EVALUATION LOGIC GOES HERE##


## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [196]:
# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(test_genes), 'Each gene should have a unique predicted expression'

NameError: name 'pred' is not defined

In [ ]:
model.eval()  # Set the model to evaluation mode
preds = []
for X_chr, y_chr in zip(train_x, train_y):
    # Convert numpy arrays to torch tensors
    X_chr = torch.tensor(X_chr.T, dtype=torch.float32).unsqueeze(0)  # (1, 2, num_genes)
    y_chr = torch.tensor(y_chr, dtype=torch.float32).unsqueeze(0)    # (1, num_genes)

    preds.append(model(X_chr))


In [ ]:
# Convert to numpy if tensor
pred_values = preds[0].detach().cpu().numpy().ravel()  # assuming same chromosome index
gene_names_chr = train_names[0]  # assuming same chromosome index

# Build DataFrame
df_pred = pd.DataFrame({
    'gene_name': gene_names_chr,
    'prediction': pred_values
})

print(df_pred)

     gene_name   prediction
0       PRSS56  1251079.250
1        MZT2B  2166804.250
2         POMC -7441639.000
3      TMEM177  2736535.750
4        TTC7A -3640067.500
...        ...          ...
1198     CASP8 -3566437.750
1199     TRPM8  3872184.750
1200   FAM166C -8041816.000
1201   DNAJC10  1045738.125
1202   SLC20A1  2441869.500

[1203 rows x 2 columns]


#### Store Predictions in the Required Format

In [3]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = 'Data/submission'  # TODO
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "Laffranchi_Paolo_Project1.zip" # TODO
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes = pd.read_csv(f"{save_dir}/{file_name}")
test_genes[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)